In [576]:
import pandas as pd
import os
import numpy as np
import xlsxwriter as xw
import re
import warnings
warnings.filterwarnings('ignore')
import openpyxl as op
from openpyxl import Workbook, load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from pathlib import Path
import glob
from google.cloud import storage
import requests
import datetime
from datetime import date

In [577]:
Month = "Jul"
Year = "2023"
Date = "16"
file_size = 24999
batch_no_1 = 1

In [578]:
input_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Target/"
output_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Result/"
input_loc_bonus_pen = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/OB Transaction/" 
result_loc_true_false = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/OB Transaction/{Year}/{Month}/{Date}/"

In [579]:
Today_date = pd.to_datetime(date.today())

In [580]:
def Create_directory(Directory_to_Check):
    if os.path.exists(Directory_to_Check):
        print("Path is Available")
    else: 
        os.makedirs(Directory_to_Check)
        print("Path is Created")

In [581]:
def Check_if_empty(Directroy_location):
    filelist = glob.glob(Directroy_location+"*", recursive=True)
    if len(filelist) == 0:
        print("File is Empty")
    else:
        for file in filelist:
            os.remove(file)

In [582]:
def Check_New_City(Table_to_Check, Column_to_filter, Column_to_Verify):
    Blank_Analysis = pd.DataFrame()
    Blank_Analysis = Table_to_Check[~Table_to_Check[Column_to_filter].isin([np.nan])]
    len(Blank_Analysis[Blank_Analysis[Column_to_Verify].isin([np.nan])])
    if(len(Blank_Analysis[Blank_Analysis[Column_to_Verify].isin([np.nan])]) > 0):
        print(f"Their is a new City in the Payout Model Update the City in the Excel & Check the file {output_loc}/Split OB/Data_For_New_City.csv")
        Blank_Analysis[Blank_Analysis[Column_to_Verify].isin([np.nan])].to_csv(output_loc+"/Split OB/"+"Data_For_New_City.csv", index=False)
        return
    else:
        print("No New City is added in Payout Model")

In [583]:
#Table_to_Update['Diff_in_day'] = Table_to_Update['Diff_in_day'].dt.days
def substract_day(Table_to_Update, Date_column, Column_to_be_replace):
    Table_to_Update['Diff_in_day'] = (Today_date-pd.to_datetime(Table_to_Update[Date_column])).dt.days
    Table_to_Update['New_Time_Remaining'] = Table_to_Update[Column_to_be_replace] - Table_to_Update['Diff_in_day']
    Table_to_Update['New_Time_Remaining_2'] = Table_to_Update['New_Time_Remaining'].apply(lambda x:0 if x<0 else x)
    Table_to_Update.drop([Column_to_be_replace,'Diff_in_day' ,'New_Time_Remaining'], axis =1, inplace=True)
    Table_to_Update.rename(columns={'New_Time_Remaining_2':Column_to_be_replace}, inplace=True)
    return Table_to_Update.head(1)

In [584]:
bonus_penalties = pd.read_excel(input_loc_bonus_pen+"Transaction Bonus and Penalties - 15th July updated.xlsx")

In [585]:
bonus_penalties['City'] = bonus_penalties['City'].str.title()

In [586]:
filelist = glob.glob(input_loc+"OB/**/*.csv",recursive=True)
filelist

['/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Target/OB/18-Jul-2023 to 19-Jul-2023.csv']

In [587]:
Outcome_con = pd.DataFrame()

for i in filelist:
    df = pd.read_csv(i)
    Outcome_con = Outcome_con.append(df)

In [588]:
Outcome_con_f = Outcome_con[['Merchant_External_Id', 'Time_Remaining', 'Outcome_Status', 'Merchant_ID', 'CityName', 'DataDate']]

In [589]:
Outcome_con_f['CityName'] = Outcome_con_f['CityName'].str.title()

In [590]:
Outcome_con_f.sort_values(by=['Outcome_Status' ,'DataDate','Time_Remaining'], ascending=[False,False,True ], inplace=True)

In [591]:
Outcome_con_f['DataDate'].unique()

array(['Jul 19, 2023', 'Jul 18, 2023'], dtype=object)

In [592]:
Outcome_con_f2 = Outcome_con_f.drop_duplicates(subset=['Merchant_External_Id'])

In [593]:
Outcome_con_f2['DataDate'] = pd.to_datetime(Outcome_con_f2['DataDate'])
Outcome_con_f2['DataDate'] = Outcome_con_f2['DataDate'].dt.strftime("%d-%b-%Y")

In [594]:
bonus_penalties.rename(columns={'City':'CityName'}, inplace=True)

In [595]:
Outcome_con_f2_merge = Outcome_con_f2.merge(bonus_penalties, on='CityName', how="left")
Outcome_con_f2_merge.drop_duplicates(subset=['Merchant_External_Id'], inplace=True)

In [596]:
len(Outcome_con_f2_merge)

59732

# Create Folder for OB

In [597]:
Create_directory(result_loc_true_false)

Path is Available


In [598]:
Check_New_City(Outcome_con_f2_merge, "Merchant_ID", "Transaction Bonus INR\n(Per OB)")

No New City is added in Payout Model


In [599]:
Outcome_con_f2_merge.to_csv(output_loc+"/Split OB/"+"True_False.csv", index=False)
Outcome_con_f2_merge.to_csv(result_loc_true_false+"True_False.csv", index=False)

In [600]:
#Stop =

In [601]:
input_loc_split = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Result/Split OB/"
result_loc_true = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/OB Transaction/{Year}/{Month}/{Date}/True/"
result_loc_false_0 = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/OB Transaction/{Year}/{Month}/{Date}/False/0/"
result_loc_false_1 = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/OB Transaction/{Year}/{Month}/{Date}/False/>0/"

In [602]:
outcome_true_false = pd.read_csv(input_loc_split+"True_False.csv")

In [603]:
substract_day(outcome_true_false, "DataDate", "Time_Remaining")

,Merchant_External_Id,Outcome_Status,Merchant_ID,CityName,DataDate,Type,Transaction Bonus INR\n(Per OB),Transaction Penalty INR\n(Per OB),Transaction Bonus INR\n(Per Inactive Revisit),Time_Remaining
0,BCR2DN4TRKMIPDK3,True,9531728.0,Pathanamthitta,19-Jul-2023,A/Exception,15.0,-40.0,NaN,0


In [604]:
outcome_true_false2 = outcome_true_false[~outcome_true_false['Merchant_ID'].isin([np.nan])]

In [605]:
outcome_true_false3 = outcome_true_false2[['Merchant_ID', 'Transaction Bonus INR\n(Per OB)', 'Transaction Penalty INR\n(Per OB)', 'Time_Remaining',
'Outcome_Status']]

In [606]:
outcome_true = outcome_true_false3[outcome_true_false3['Outcome_Status'].isin([True])]
outcome_false = outcome_true_false3[outcome_true_false3['Outcome_Status'].isin([False])]
outcome_false_0 = outcome_false[outcome_true_false3['Time_Remaining'].isin([0])]
outcome_false_1 = outcome_false[~outcome_true_false3['Time_Remaining'].isin([0])]

In [607]:
outcome_true_f = outcome_true[['Merchant_ID','Transaction Bonus INR\n(Per OB)', 'Time_Remaining']].rename(
    columns={'Merchant_ID':'Merchant ID','Transaction Bonus INR\n(Per OB)':'Comment', 'Time_Remaining':'Remaining Days'})

In [608]:
outcome_true_f['Transaction Category'] = "Transaction Complete - No Action"
outcome_true_f['Comment'] = outcome_true_f['Comment'].astype(int)
outcome_true_f['Comment'] = outcome_true_f['Comment'].astype(str)
#outcome_true_f['Comment'] = outcome_true_f['Comment'].str.replace(".0","")
outcome_true_f['New Comment'] = "+"+outcome_true_f['Comment']+" INR"

In [609]:
outcome_true_f2 = outcome_true_f[['Merchant ID', 'Transaction Category', 'New Comment', 'Remaining Days']].rename(columns={'New Comment':'Comment'})
outcome_true_f2.to_csv(input_loc_split+"/True/True.csv", index=False)

In [610]:
outcome_false_0_f = outcome_false_0[['Merchant_ID','Transaction Penalty INR\n(Per OB)', 'Time_Remaining']].rename(
    columns={'Merchant_ID':'Merchant ID','Transaction Penalty INR\n(Per OB)':'Comment', 'Time_Remaining':'Remaining Days'})

In [611]:
outcome_false_0_f['Transaction Category'] = "Transaction Missed (14 day cycle missed)"
outcome_false_0_f['Comment'] = outcome_false_0_f['Comment'].astype(int)
outcome_false_0_f['Comment'] = outcome_false_0_f['Comment'].astype(str)
#outcome_false_0_f['Comment'] = outcome_false_0_f['Comment'].str.replace(".0","")
outcome_false_0_f['New Comment'] = outcome_false_0_f['Comment']+" INR"

In [612]:
outcome_false_0_f2 = outcome_false_0_f[['Merchant ID', 'Transaction Category', 'New Comment', 'Remaining Days']].rename(columns={'New Comment':'Comment'})
outcome_false_0_f2.to_csv(input_loc_split+"False/0/False_0.csv", index=False)

In [613]:
outcome_false_1_f = outcome_false_1[['Merchant_ID','Transaction Penalty INR\n(Per OB)', 'Time_Remaining']].rename(
    columns={'Merchant_ID':'Merchant ID','Transaction Penalty INR\n(Per OB)':'Comment', 'Time_Remaining':'Remaining Days'})

In [614]:
outcome_false_1_f['Transaction Category'] = "Transaction Incomplete (14 days) - Action Needed"
outcome_false_1_f['Comment'] = outcome_false_1_f['Comment'].astype(int)
outcome_false_1_f['Comment'] = outcome_false_1_f['Comment'].astype(str)
#outcome_false_1_f['Comment'] = outcome_false_1_f['Comment'].str.replace(".0","")
outcome_false_1_f['New Comment'] = outcome_false_1_f['Comment']+" INR"

In [615]:
outcome_false_1_f2 = outcome_false_1_f[['Merchant ID', 'Transaction Category', 'New Comment', 'Remaining Days']].rename(columns={'New Comment':'Comment'})
outcome_false_1_f2.to_csv(input_loc_split+"/False/>0/False>0.csv", index=False)

In [616]:
Create_directory(result_loc_true)

Path is Available


In [617]:
Check_if_empty(result_loc_true)

In [618]:
ob_final = pd.DataFrame()
for j in pd.read_csv(input_loc_split+"/True/True.csv", chunksize=file_size):
    j['Remaining Days'] = j['Remaining Days'].apply(lambda x: x if x < 0 else 0)
    j.to_excel(result_loc_true+'OB_MerchantTransactionData_Format'+str(batch_no_1)+'.xlsx', index=False)
    ob_final = ob_final.append(j)
    batch_no_1 += 1

In [619]:
Create_directory(result_loc_false_0)

Path is Available


In [620]:
Check_if_empty(result_loc_false_0)

In [621]:
for i in pd.read_csv(input_loc_split+"/False/0/False_0.csv", chunksize=file_size):
    i.to_excel(result_loc_false_0+'OB_MerchantTransactionData_Format'+str(batch_no_1)+'.xlsx', index=False)
    ob_final = ob_final.append(i)
    batch_no_1 += 1

In [622]:
Create_directory(result_loc_false_1)

Path is Available


In [623]:
Check_if_empty(result_loc_false_1)

In [624]:
for k in pd.read_csv(input_loc_split+"/False/>0/False>0.csv", chunksize=file_size):
    k.to_excel(result_loc_false_1+'OB_MerchantTransactionData_Format'+str(batch_no_1)+'.xlsx', index=False)
    ob_final = ob_final.append(k)
    batch_no_1 += 1

In [625]:
ob_final.to_csv(result_loc_true_false + "Final_OB_File.csv", index=False)

In [626]:
#Stop =

# Revisit Consolidate

In [627]:
input_loc = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Target/Revisit/"
output_loc_split_revisit = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Result/Split Revisit/"
output_loc_final_revisit = "/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/OB Transaction/Sample/Revisit/"
result_loc_true_false = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Revisit Transaction/{Year}/{Month}/{Date}/"

In [628]:
Create_directory(result_loc_true_false)

Path is Available


In [629]:
filelist = glob.glob(input_loc+"*/**/*.csv", recursive=True)

In [630]:
revisit_outcome = pd.DataFrame()
for i in filelist:
    df = pd.read_csv(i)
    revisit_outcome = revisit_outcome.append(df)

In [631]:
revisit_outcome.head(1)

,merchant_external_id,region,state,district,postal_code,revisit_timestamp,time_remaining,Outcome_Status,City Name,Merchant ID,...,POD Name,POD ID,Revisit Date,Remarks,Team Lead Name,CM,Cluster,Report_Date,outcome_status,Role Name\n
0,BCR2DN6T46GKHWKK,West,MAHARASHTRA,Yavatmal,445105,2023-06-28,0,True,Yavatmal,6536119.0,...,Mahesh Rajusing Rathod,33291.0,28/06/2023,Revisit Done,Vishal.Dabhade,Sudeep,Pune,2023-07-16,NaN,NaN


In [632]:
revisit_outcome.rename(columns={'Merchant_ID':'Merchant ID', 'Region':'region', 'MerchantExternalId':'merchant_external_id', 'State':'state', 'District':'district',
                                'Pincode':'postal_code', 'Revisit_Date':'revisit_timestamp', 'Time_Remaining':'time_remaining', 'Datadate':'Report_Date'}, inplace=True)

In [633]:
revisit_outcome.columns

Index(['merchant_external_id', 'region', 'state', 'district', 'postal_code',
       'revisit_timestamp', 'time_remaining', 'Outcome_Status', 'City Name',
       'Merchant ID', 'Merchant Name', 'On Boarding Date',
       'On-Boarded Employee Code', 'Employee Code', 'Role Name', 'POD Name',
       'POD ID', 'Revisit Date', 'Remarks', 'Team Lead Name', 'CM', 'Cluster',
       'Report_Date', 'outcome_status', 'Role Name\n'],
      dtype='object')

In [634]:
revisit_outcome['Outcome_Status'].fillna(revisit_outcome['outcome_status'], inplace=True)

In [635]:
revisit_outcome_f = revisit_outcome[['merchant_external_id', 'region', 'state', 'district', 'postal_code', 'revisit_timestamp', 'time_remaining', 
'Outcome_Status', 'Report_Date', 'Merchant ID']]

In [636]:
revisit_outcome_f2 = revisit_outcome_f[~revisit_outcome_f['Merchant ID'].isin([np.nan])]

In [637]:
revisit_outcome_f2.to_csv(output_loc_split_revisit+"Revisit_True_False.csv", index=False)
revisit_outcome_f2.to_csv(result_loc_true_false+"Revisit_True_False.csv", index=False)

In [638]:
#Revisit_stop = 

In [639]:
update_revisit_outcome = pd.read_csv(output_loc_split_revisit+"Revisit_True_False.csv")

In [640]:
result_loc_true = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Revisit Transaction/{Year}/{Month}/{Date}/True/"
result_loc_false_0 = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Revisit Transaction/{Year}/{Month}/{Date}/False/0/"
result_loc_false_1 = f"/home/dhrubakumardolai/Desktop/Python Test/Rahul/Gpay Outcome OB Transactions/Revisit Transaction/{Year}/{Month}/{Date}/False/>0/"

In [641]:
Create_directory(result_loc_true)
Create_directory(result_loc_false_0)
Create_directory(result_loc_false_1)
Create_directory(result_loc_true+"Invalid/")
Create_directory(result_loc_false_0+"Invalid/")
Create_directory(result_loc_false_1+"Invalid/")

Path is Available
Path is Available
Path is Available


In [642]:
Check_if_empty(result_loc_true)
Check_if_empty(result_loc_false_0)
Check_if_empty(result_loc_false_1)

In [643]:
outcome_true_false = update_revisit_outcome

In [644]:
substract_day(outcome_true_false, "Report_Date", 'time_remaining')

,merchant_external_id,region,state,district,postal_code,revisit_timestamp,Outcome_Status,Report_Date,Merchant ID,time_remaining
0,BCR2DN6T46GKHWKK,West,MAHARASHTRA,Yavatmal,445105,2023-06-28,True,2023-07-16,6536119.0,0


In [645]:
outcome_true_false.rename(columns={'merchant_external_id':'Merchant_External_Id', 'time_remaining':'Time_Remaining', 'Outcome_Status':'Outcome_Status',
'Report_Date':'DataDate', 'Merchant ID':'Merchant_ID', 'district':'CityName', 'Merchant ID':'Merchant_ID'}, inplace=True)
outcome_true_false['Transaction Bonus INR\n(Per OB)'] = np.nan
outcome_true_false['Transaction Bonus INR\n(Per OB)'].replace(np.nan, 'NA', inplace=True)
outcome_true_false['Transaction Penalty INR\n(Per OB)'] = np.nan
outcome_true_false['Transaction Penalty INR\n(Per OB)'].replace(np.nan, 'NA', inplace=True)

In [646]:
outcome_true_false.sort_values(by=['Outcome_Status','DataDate','Time_Remaining'], ascending=[False, False, True], inplace=True)
outcome_true_false.drop_duplicates(subset=['Merchant_ID'], inplace=True)

In [647]:
len(outcome_true_false)

92792

In [648]:
outcome_true_false2 = outcome_true_false[~outcome_true_false['Merchant_ID'].isin([np.nan])]

In [649]:
outcome_true_false3 = outcome_true_false2[['Merchant_ID', 'Transaction Bonus INR\n(Per OB)', 'Transaction Penalty INR\n(Per OB)', 'Time_Remaining',
'Outcome_Status']]

In [650]:
outcome_true = outcome_true_false3[outcome_true_false3['Outcome_Status'].isin([True])]
outcome_false = outcome_true_false3[outcome_true_false3['Outcome_Status'].isin([False])]
outcome_false_0 = outcome_false[outcome_true_false3['Time_Remaining'].isin([0])]
outcome_false_1 = outcome_false[~outcome_true_false3['Time_Remaining'].isin([0])]

In [651]:
outcome_true_f = outcome_true[['Merchant_ID','Transaction Bonus INR\n(Per OB)', 'Time_Remaining']].rename(
    columns={'Merchant_ID':'Merchant ID','Transaction Bonus INR\n(Per OB)':'Comment', 'Time_Remaining':'Remaining Days'})

In [652]:
outcome_true_f['Transaction Category'] = "Transaction Complete - No Action"
outcome_true_f['Comment'] = outcome_true_f['Comment'].astype(str)
outcome_true_f['Comment'] = outcome_true_f['Comment'].str.replace(".0","")
outcome_true_f['New Comment'] = ""+outcome_true_f['Comment']+""

In [653]:
outcome_true_f2 = outcome_true_f[['Merchant ID', 'Transaction Category', 'New Comment', 'Remaining Days']].rename(columns={'New Comment':'Comment'})
outcome_true_f2.to_csv(output_loc+"/Split Revisit/True/True.csv", index=False)

In [654]:
outcome_false_0_f = outcome_false_0[['Merchant_ID','Transaction Penalty INR\n(Per OB)', 'Time_Remaining']].rename(
    columns={'Merchant_ID':'Merchant ID','Transaction Penalty INR\n(Per OB)':'Comment', 'Time_Remaining':'Remaining Days'})

In [655]:
outcome_false_0_f['Transaction Category'] = "Transaction Missed (14 day cycle missed)"
outcome_false_0_f['Comment'] = outcome_false_0_f['Comment'].astype(str)
outcome_false_0_f['Comment'] = outcome_false_0_f['Comment'].str.replace(".0","")
outcome_false_0_f['New Comment'] = outcome_false_0_f['Comment']+""

In [656]:
outcome_false_0_f2 = outcome_false_0_f[['Merchant ID', 'Transaction Category', 'New Comment', 'Remaining Days']].rename(columns={'New Comment':'Comment'})
outcome_false_0_f2.to_csv(output_loc+"/Split Revisit/False/0/False_0.csv", index=False)


In [657]:
outcome_false_1_f = outcome_false_1[['Merchant_ID','Transaction Penalty INR\n(Per OB)', 'Time_Remaining']].rename(
    columns={'Merchant_ID':'Merchant ID','Transaction Penalty INR\n(Per OB)':'Comment', 'Time_Remaining':'Remaining Days'})

In [658]:
outcome_false_1_f['Transaction Category'] = "Transaction Incomplete (14 days) - Action Needed"
outcome_false_1_f['Comment'] = outcome_false_1_f['Comment'].astype(str)
outcome_false_1_f['Comment'] = outcome_false_1_f['Comment'].str.replace(".0","")
outcome_false_1_f['New Comment'] = outcome_false_1_f['Comment']+""

In [659]:
outcome_false_1_f2 = outcome_false_1_f[['Merchant ID', 'Transaction Category', 'New Comment', 'Remaining Days']].rename(columns={'New Comment':'Comment'})
outcome_false_1_f2.to_csv(output_loc+"/Split Revisit/False/>0/False>0.csv", index=False)

In [660]:
Create_directory(result_loc_true_false+"/Revisit Target/")

Path is Available


In [661]:
Revisit_final = pd.DataFrame()
for j in pd.read_csv(output_loc+"/Split Revisit/True/True.csv", chunksize=file_size):
    j['Comment'].replace(np.nan,"NA", inplace=True) 
    j['Remaining Days'] = j['Remaining Days'].apply(lambda x: x if x < 0 else 0)
    j.to_excel(result_loc_true+'RV_MerchantTransactionData_Format'+str(batch_no_1)+'.xlsx', index=False)
    Revisit_final = Revisit_final.append(j)
    batch_no_1 += 1

In [662]:
for i in pd.read_csv(output_loc+"/Split Revisit/False/0/False_0.csv", chunksize=file_size):
    i['Comment'].replace(np.nan,"NA", inplace=True)
    i.to_excel(result_loc_false_0+'RV_MerchantTransactionData_Format'+str(batch_no_1)+'.xlsx', index=False)
    Revisit_final = Revisit_final.append(i)
    batch_no_1 += 1

In [663]:
for k in pd.read_csv(output_loc+"/Split Revisit/False/>0/False>0.csv", chunksize=file_size):
    k['Comment'].replace(np.nan,"NA", inplace=True)
    k.to_excel(result_loc_false_1+'RV_MerchantTransactionData_Format'+str(batch_no_1)+'.xlsx', index=False)
    Revisit_final = Revisit_final.append(k)
    batch_no_1 += 1

In [664]:
Revisit_final.to_csv(result_loc_true_false+'Revisit_final_file.csv', index=False)